In [1]:
%load_ext autoreload
%autoreload 2

# 1. Bài toán minh họa

![](./images/00.png)<br>


# 3. Xây dựng model

In [2]:
from modules.my_spark_regression import *
from modules.my_pyspark import *
from modules.my_drawer import MyDrawer

In [3]:
spark = MyPySpark(session=True, sql=True)
drawer = MyDrawer()

## 3.1. Chuẩn bị & chuẩn hóa dữ liệu, xác định input, output

* Đọc dữ liệu

In [4]:
file_path = r'./data/Ecommerce_Customers.csv'

In [5]:
data = spark.readFile(file_path)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



> * Các thuộc tính mơ hồ:
>   * `Avg Session Length`: thời gian trung bình mà khách hàng sử dụng web hoặc app của công ty.
>   * `Time on App`: thời gian dùng trên app
>   * `Time on Website`: thời gian dùng trên web
>   * `Length of Membership`: thời gian mất bao lâu để khách hàng trở thành thành viên của công ty
>   * `Yearly Amount Spent`: số tiền mà khách hàng sẽ **cúng** cho công ty trong năm

In [7]:
data.head()

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)

* Xác định input và output
  * Các thuộc tính input: `Avg Session Length`, `Time on App`, `Time on Website`, `Length of Membership`.
  * Thuộc tính dự đoán: `Yearly Amount Spent`.

In [8]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [9]:
input_features = [
    'Avg Session Length',
    'Time on App',
    'Time on Website',
    'Length of Membership'
]

* Từ các thuộc tính trong `input_features` mình sẽ combine nó lại và tạo ra một vector duy nhất gọi là `features` (dc chỉ định tại tham số **outputCol**).

In [10]:
assembler = VectorAssembler(inputCols=input_features, outputCol='features') # đây là input

In [11]:
data_pre = assembler.transform(data) # transform `data` để khớp vs `assembler`

* Lúc này ta có thể gọi thuộc tính `features` đại diện cùng lúc cho 4 thuộc tính mà ta đã định nghĩa tại `input_features`.

In [12]:
data_pre.select('features').show(2, False) 

+--------------------------------------------------------------------------+
|features                                                                  |
+--------------------------------------------------------------------------+
|[34.49726772511229,12.65565114916675,39.57766801952616,4.0826206329529615]|
|[31.92627202636016,11.109460728682564,37.268958868297744,2.66403418213262]|
+--------------------------------------------------------------------------+
only showing top 2 rows



* Tạo ra tập dữ liệu cuồi cùng gọi là `final_data` dùng để training model với input là `features` và output là `Yearly Amount Spent`.

In [13]:
final_data = data_pre.select('features', 'Yearly Amount Spent')

In [14]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

## 3.2. Chuẩn bị train/test data

In [15]:
train_data, test_data = final_data.randomSplit((0.7, 0.3)) # 0.7 train và 0.3 test

* Xem thống kê mô tả trên training data và test data.

In [16]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                365|
|   mean| 500.62556223304955|
| stddev|  79.90015064991094|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [17]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                135|
|   mean| 495.76806603134975|
| stddev|  77.89261335663029|
|    min|  275.9184206503857|
|    max|  684.1634310159512|
+-------+-------------------+



> * Dữ liệu train và test gần như tương đương, ko có sự chênh lệch cao về mặt thống kê.
> * Nếu chênh lệch giữa các giá trị thống kê của training data và test data là $\leq 10\%$ thì OKLA.

## 3.3. Xây dựng model

* Tạo model Linear Regression
* Tham số `predictionCol` là giá trị mà model sẽ dự đoán, tức $\widehat{y}$

In [18]:
lr = LinearRegression(featuresCol='features', labelCol='Yearly Amount Spent', predictionCol='predict_Yearly Amount Spent') # 

* Fit model với data và gán model cho một biến nào đó

In [19]:
lrModel = lr.fit(train_data)

* In ra coefficients và intercept

In [20]:
lrModel.coefficients, lrModel.intercept

(DenseVector([25.713, 38.6829, -0.1203, 61.2778]), -1028.6962111775113)

## 3.4. Đánh giá model vs test data

In [21]:
test_results = lrModel.evaluate(test_data)

* Đánh giá phần dư

In [22]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  10.50012861084042|
| -5.121812427462771|
|  6.258832743758603|
| -22.29634497612176|
| -7.094239509159706|
| -5.135244488962371|
| 2.1382327092534297|
|  3.514817845767652|
| 0.8242960148279508|
| 3.0395071779875593|
| -4.328748621591842|
| -8.659823607413216|
| -5.405240617714128|
|  6.669525669209122|
| -5.143398546796334|
|  7.364276618996712|
| -6.153407498195008|
| -9.300333269112286|
|-17.382429922941867|
| 11.472059407789231|
+-------------------+
only showing top 20 rows



* Đánh giá RMSE

In [23]:
test_results.rootMeanSquaredError

9.504243094543694

* Đánh giá mean squared error

In [24]:
test_results.meanSquaredError

90.33063680018151

* Đánh giá $R^2$

In [25]:
test_results.r2

0.9850006827140068

## 3.5. Đánh giá model vs test data

In [26]:
test_model = lrModel.transform(test_data)
test_model.select('predict_Yearly Amount Spent', 'Yearly Amount Spent').show()

+---------------------------+-------------------+
|predict_Yearly Amount Spent|Yearly Amount Spent|
+---------------------------+-------------------+
|         398.14022246178706|  408.6403510726275|
|          472.6237128544524|  467.5019004269896|
|          488.3797770131341|  494.6386097568927|
|         509.24339881588753|  486.9470538397658|
|          564.3469262562144|  557.2526867470547|
|          428.6057776627863|  423.4705331738239|
|          430.5824851306802|  432.7207178399336|
|         491.66113260370776|  495.1759504494754|
|         409.24531504515494|  410.0696110599829|
|           481.837457757141| 484.87696493512857|
|         280.24716927197755|  275.9184206503857|
|           417.754349799751|  409.0945261923378|
|          381.7421413746383|  376.3369007569242|
|          468.5938980583394|  475.2634237275485|
|         397.95374353059356|  392.8103449837972|
|          465.6279700478017|  472.9922466667984|
|          398.4386517444625|  392.2852442462675|


## 3.6. Lưu trữ & tải model

* Lưu model

In [27]:
file_path1 = r'./data/lrModel_Ecommerce_Customers'

In [28]:
lrModel.save(file_path1)

Py4JJavaError: An error occurred while calling o220.save.
: java.io.IOException: Path ./data/lrModel_Ecommerce_Customers already exists. To overwrite it, please use write.overwrite().save(path) for Scala and use write().overwrite().save(path) for Java and Python.
	at org.apache.spark.ml.util.FileSystemOverwrite.handleOverwrite(ReadWrite.scala:683)
	at org.apache.spark.ml.util.MLWriter.save(ReadWrite.scala:167)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at java.base/jdk.internal.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at java.base/jdk.internal.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.base/java.lang.reflect.Method.invoke(Method.java:566)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(Abstr

* Tải model

In [32]:
from pyspark.ml.regression import LinearRegressionModel

In [33]:
lrModel2 = LinearRegressionModel.load(file_path1)

## 3.7. Dự đoán dữ liệu mới

In [34]:
unlabeled_data = test_data.select('features')
preditions = lrModel2.transform(unlabeled_data)

In [36]:
preditions.show()

+--------------------+---------------------------+
|            features|predict_Yearly Amount Spent|
+--------------------+---------------------------+
|[30.4925366965402...|          287.9716876515006|
|[30.7377203726281...|          451.4531490841305|
|[30.8364326747734...|         472.37039264151827|
|[31.1239743499119...|         508.84162193564316|
|[31.2606468698795...|          422.7389670833363|
|[31.2834474760581...|          569.9208634421666|
|[31.3584771924370...|         491.53583276578183|
|[31.4252268808548...|          534.8952953365547|
|[31.5147378578019...|         494.18800585094846|
|[31.6253601348306...|         381.70254780395385|
|[31.7242025238451...|         510.31043913621625|
|[31.8293464559211...|         384.06433874584786|
|[31.8627411090001...|          558.8838258513274|
|[31.9262720263601...|         380.07417041060967|
|[31.9453957483445...|          662.5628115177722|
|[31.9563005605233...|          565.1072644299811|
|[31.9764800614612...|         